In [7]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#MNIST 다운로드
mnist      = input_data.read_data_sets('/tensorflow/mnistdata', one_hot=True) #숫자 하나만 선택되도록

Extracting /tensorflow/mnistdata\train-images-idx3-ubyte.gz
Extracting /tensorflow/mnistdata\train-labels-idx1-ubyte.gz
Extracting /tensorflow/mnistdata\t10k-images-idx3-ubyte.gz
Extracting /tensorflow/mnistdata\t10k-labels-idx1-ubyte.gz


In [8]:
# Hyper Parameters
learning_rate = 0.05
training_epochs = 20
batch_size = 100
display_step = 1
logs_path = '/deep/tensorflow/log'

# tf Graph Input
x_input = tf.placeholder(tf.float32, [None, 784], name='InputData') # MNIST 이미지 사이즈 28*28=784
y_label = tf.placeholder(tf.float32, [None, 10], name='LabelData')  # 0-9 범위안의 답 => 10 classes

# Weight와 Bias 초기화 
W1 = tf.Variable(tf.random_normal([784, 100], mean=0.0, stddev=1.0))
b1 = tf.Variable(tf.zeros([100]))
W2 = tf.Variable(tf.random_normal([100, 10], mean=0.0, stddev=1.0))
b2 = tf.Variable(tf.zeros([10]))

with tf.name_scope('Model'):
    hidden1 = tf.sigmoid(tf.matmul(x_input, W1) + b1)
    predict = tf.matmul(hidden1, W2) + b2 #분류를 위해 Softmax가 필요하나, 아래 cost함수에서 함께해 줌
    
with tf.name_scope('Loss'):
    cross_entropy_cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels= y_label, logits= predict))
    
with tf.name_scope('SGD'): # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy_cost)
    
with tf.name_scope('Accuracy'): # 정확도
    acc = tf.equal(tf.argmax(predict, 1), tf.argmax(y_label, 1)) #prediction과 label이 같은지 비교
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))
    
tf.summary.histogram("x_input", x_input)
tf.summary.histogram("predict", predict)
tf.summary.histogram("y_label", y_label)
tf.summary.scalar("loss", cross_entropy_cost)
tf.summary.scalar("accuracy", acc)

merged_summary_op = tf.summary.merge_all() #모든 summary 합치기

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Tensorboard에 쓰기
    summary_writer = tf.summary.FileWriter(logs_path, graph=sess.graph)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)

        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)

            _, c, summary = sess.run([optimizer, cross_entropy_cost, merged_summary_op],
                                     feed_dict={x_input: batch_xs, y_label: batch_ys})

            summary_writer.add_summary(summary, epoch * total_batch + i)

            avg_cost += c / total_batch
        accuracy = sess.run(acc, feed_dict={x_input: mnist.test.images, y_label: mnist.test.labels})
        print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost), "정확도:", "{:.9f}".format(accuracy)) #진행상황 출력

    print("학습 끝!")
    
    print("run_tensorboard.bat 실행!" , "\n브라우저에서 http://localhost:6006/ 에 접속")

InvalidArgumentError: You must feed a value for placeholder tensor 'InputData' with dtype float and shape [?,784]
	 [[node InputData (defined at d:\deep\python\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]

Original stack trace for 'InputData':
  File "d:\deep\python\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\deep\python\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\deep\python\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\deep\python\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "d:\deep\python\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
  File "d:\deep\python\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "d:\deep\python\lib\asyncio\base_events.py", line 534, in run_forever
